In [32]:
%load_ext autoreload
%autoreload

import pathlib
from lam4vb3 import build
import pandas as pd
from lam4vb3 import property_build
from lam4vb3 import class_build
from lam4vb3 import lam_utils
import rdflib
from rdflib.namespace import RDF, RDFS, SKOS, DCTERMS, OWL, XMLNS, XSD

INPUT_FILE = pathlib.Path("../docs/semi-structured/LAM_metadata_03.xlsx").resolve()
OUTPUT_FILE = pathlib.Path("../output/lam_project_v1.ttl").resolve()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
lam_df_properties = pd.read_excel(INPUT_FILE, sheet_name=0 , header = [0], na_values=[""], keep_default_na=False )
lam_df_classes = pd.read_excel(INPUT_FILE, sheet_name=1 ,header = [0], na_values=[""], keep_default_na=False )
celex_df_properties = pd.read_excel(INPUT_FILE, sheet_name=2 ,header = [0], na_values=[""], keep_default_na=False )
celex_df_classes = pd.read_excel(INPUT_FILE, sheet_name=3 ,header = [0], na_values=[""], keep_default_na=False )
prefixes = pd.read_excel(INPUT_FILE, sheet_name=4 ,header = [0], na_values=[""], keep_default_na=False )
# test = lam_df_properties.head().to_dict()
# print(str(test))


In [33]:
# property_build.make_property_worksheet(lam_df_properties,prefixes, OUTPUT_FILE)

In [43]:

LITERAL_COLUMNS = {
    'EXAMPLE_EN': 'skos:example@en',
    'EXAMPLE_FR': 'skos:example@fr',
    'COMMENT': 'skos:editorialNote@en',
    'EXAMPLE_CELEX': 'skos:example',
}

REIFIED_LITERAL_COLUMNS = {
    'KEYWORD': 'skosxl:literalForm@en',
}

MAPPING_URI_COLUMNS = {
    'CDM_CLASS': 'lam:cdm_class',
    'DN_CLASS': 'lam:celex_class',
}

MAPPING_VALUE_COMMENT_COLUMNS = {
    'AU': 'cdm:created_by',
    'FM': 'cdm:resource-type',
}

MAPPING_CELEX_COLUMN = {
    'DN_CLASS': 'lam:celex_class',
}

URI_COLUMN = 'URI'


graph = build.make_graph(prefixes) 

df = lam_df_classes
# make literal columns
literal_maker = build.MultiColumnTripleMaker(df,
                                             subject_source=URI_COLUMN,
                                             subject_class="skos:Concept",
                                             column_mapping_dict=LITERAL_COLUMNS,
                                             target_columns=list(LITERAL_COLUMNS.keys()),
                                             uri_valued_columns=[],
                                             multi_line_columns=[],
                                             graph=graph)
literal_maker.make_triples()

# make literal columns
reified_literal_maker = build.MultiColumnTripleMaker(df,
                                             subject_source=None,
                                             subject_class="skosxl:Label",
                                             column_mapping_dict=REIFIED_LITERAL_COLUMNS,
                                             target_columns=list(REIFIED_LITERAL_COLUMNS.keys()),
                                             uri_valued_columns=[],
                                             multi_line_columns=[],
                                             graph=graph)
reified_literal_maker.make_triples()
# adding reified labels
build.relate_subject_sets(literal_maker.subject_index(), reified_literal_maker.subject_index(),
                         graph=graph, predicate="skosxl:prefLabel")

# make mapping uri columns
uri_maker = build.MultiColumnTripleMaker(df,
                                             subject_source=URI_COLUMN,
                                             subject_class="skos:Concept",
                                             column_mapping_dict=MAPPING_URI_COLUMNS,
                                             target_columns=list(MAPPING_URI_COLUMNS.keys()),
                                             uri_valued_columns=list(MAPPING_URI_COLUMNS.keys()),
                                             multi_line_columns=[],
                                             graph=graph)
uri_maker.make_triples()

# make mapping commented value columns 
mapping_value_maker = build.MultiColumnTripleMaker(df,
                                             subject_source=URI_COLUMN,
                                             subject_class="skos:Concept",
                                             column_mapping_dict=MAPPING_VALUE_COMMENT_COLUMNS,
                                             target_columns=list(MAPPING_VALUE_COMMENT_COLUMNS.keys()),
                                             uri_valued_columns=[],
                                             multi_line_columns=list(MAPPING_VALUE_COMMENT_COLUMNS.keys()),
                                             graph=graph)
mapping_value_maker.make_triples()


graph.serialize(str(OUTPUT_FILE), format='turtle', )